In [2]:
!conda activate py38

In [3]:
import os
import navpy
import numpy as np
import pandas as pd
import plotly.express as px
from datetime import datetime
import plotly.graph_objects as go

In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>"
    + "#notebook { padding-top:0px !important; } " 
    + ".container { width: 95% !important; } "
    + ".end_space { min-height:0px !important; } "
    + "</style>"))

%matplotlib notebook

In [5]:
velodyne_data_path = r'C:\Users\noyl\workspace\autonomous_systems\proj1\2011_09_26_drive_0106_sync\2011_09_26\2011_09_26_drive_0106_sync\velodyne_points\data'
data_path = r'C:\Users\noyl\workspace\autonomous_systems\proj1\2011_09_26_drive_0106_sync\2011_09_26\2011_09_26_drive_0106_sync'

In [10]:
def read_velodyne_points(path):
    velodyne_data_filles = os.listdir(path)
    data_points = []
    for data_file in velodyne_data_filles:
        f = open(path + '\\' + data_file, "rb")
        data = np.fromfile(f, np.float32)
        data.shape = (-1, 4)
        data_points.append(data)
#         data_points[data_file] = data
        f.close()
        
    return data_points

velo_points = read_velodyne_points(velodyne_data_path)

In [ ]:
marker_data = go.Scatter3d(
    x=np.ceil(xs / 0.2),
    y=np.ceil(ys / 0.2), 
    z=np.ceil(zs / 0.2), 
    marker=go.scatter3d.Marker(size=3),
    mode='markers'
)
fig=go.Figure(data=marker_data)

fig.update_layout(
    width=1800,
    height=1400
)
fig.show()

In [ ]:
xs, ys = reduce_to_2d(velo_points[150][:, 0:3].copy().T, min_height=0.3, col_hits=2, res=0.2)
fig = px.scatter(x=xs, y=ys)
fig.show()

In [340]:
np.array((data['velo_xs']))

array([76.4, 61. , 60. , ..., 17.4, 17.4, 59.8], dtype=float32)

In [111]:
def read_velodyne_points(path):
    velodyne_data_filles = os.listdir(path)
    data_points = []
    for data_file in velodyne_data_filles:
        f = open(path + '\\' + data_file, "rb")
        data = np.fromfile(f, np.float32)
        data.shape = (-1, 4)
        data_points.append(data)
#         data_points[data_file] = data
        f.close()
        
    return data_points

def initialize(data_path):
    oxt_data_keys = ['lat', 'lon', 'alt', 'roll', 'pitch', 'yaw', 'vn', 'vf', 'vl', 'vu', 'ax', 'ay', 'af', 'al', 'au', 'wx', 'wy', 'wz',
                    'wf', 'wl', 'wu', 'pos_accuracy', 'vel_accuracy', 'navstat', 'numsats', 'posmode', 'velmode', 'orimode']
    # reading oxts
    oxts_files = os.listdir(data_path + '\oxts')
    oxts_times_file = data_path + '\\oxts\\timestamps.txt'
    oxts_data_path = data_path + '\\oxts\\data\\'
    oxts_data_format = data_path + '\\oxts\\dataformat.txt'
    oxts_times = []
    with open(oxts_times_file) as f:
        line = f.readline()[0:-4]
        oxts_times.append(datetime.strptime(line, '%Y-%m-%d %H:%M:%S.%f'))
        while line:
            line = f.readline()[0:-4]
            if line:
                oxts_times.append(datetime.strptime(line, '%Y-%m-%d %H:%M:%S.%f'))
                
        
    oxts_data_points = pd.DataFrame()
    data_dict = {}
    for data_file in os.listdir(oxts_data_path):
        with open(oxts_data_path + '\\' + data_file) as f:
            data_line = f.readline()
            broken_line = data_line.split(' ')
            
            for key in oxt_data_keys:
                if key not in data_dict.keys():
                    data_dict[key] = [float(broken_line.pop(0))]
                    
                else:
                    data_dict[key].append(float(broken_line.pop(0)))
        
    data = pd.DataFrame(index=oxts_times, data=data_dict)
    data['velodyne_points'] = read_velodyne_points(data_path  + '\\velodyne_points\\data')
    data = movement_correct_velo(data)
    return data

def reduce_to_2d(points, min_height=0.3, col_hits=2, res=0.2):
    # points = xs, ys, zs
    [xs, ys, zs] = points
    xs = xs[zs > min_height]
    ys = ys[zs > min_height]
    zs = zs[zs > min_height]
    xs = np.ceil(xs / res) * res
    ys = np.ceil(ys / res) * res

    points_to_keep = []
    while len(xs) > 0 and len(ys) > 0:
        x = xs[0]
        y = ys[0]
        point_bool_index = np.logical_and(xs == x, ys == y)
        count = np.sum(point_bool_index)
        if [x,y] not in points_to_keep and count > col_hits - 1:
            points_to_keep.append([x,y])

        xs = xs[np.logical_not(point_bool_index)]
        ys = ys[np.logical_not(point_bool_index)]
    
#     return np.array(points_to_keep)[:,0], np.array(points_to_keep)[:,1]
    return points_to_keep


def movement_correct_velo(data, min_height=0.3, col_hits=2, res=0.2):
    xs = []
    ys = []
    initial_lla = [data.iloc[0]['lat'], data.iloc[0]['lon'], data.iloc[0]['alt']]
    for data_point in data.iterrows():
        data_point = data_point[1]
#         print(data_point)
#         print(data_point.loc['velodyne_points'])
        points = reduce_to_2d(data_point['velodyne_points'][:, 0:3].copy().T, min_height=min_height, col_hits=col_hits, res=res)
#         [x2d, y2d] = reduce_to_2d(data_point[:, 0:3].copy().T, min_height=0.3, col_hits=2, res=0.2)
#         points = np.vstack([np.array((data['velo_xs'][i])), np.array((data['velo_ys'][i]))])
#         xs.append(x2d)
#         ys.append(y2d)
        c, s = np.cos(np.array(data_point['yaw'])), np.sin(np.array(data_point['yaw']))
        local_NE_rotation = np.array(((c, -s), (s, c)))
        current_lla = [data_point['lat'], data_point['lon'], data_point['alt']]
        NED_displacement = navpy.lla2ned(*(current_lla + initial_lla))
        NED_points = (local_NE_rotation @ np.array(points).T).T + NED_displacement[0:2]
        xs.append(NED_points[:, 0])
        ys.append(NED_points[:, 1])
        
    data['corrected_xs'] = xs
    data['corrected_ys'] = ys
    
    return data
    

# def initialize_step_grid(data_point, res, distance_func, initial_grid_location=[-1, -1]):
# # north is y, east x.
#     if -1 in initial_grid_location:
#         y_origin = np.ceil(np.abs(np.min(data_point['corrected_ys'])) / res)
#         x_origin = np.ceil(np.abs(np.min(data_point['corrected_xs'])) / res)

        
def initialize_grid(data, res):
    global_max_x = 0
    global_max_y = 0
    global_min_x = 0
    global_min_y = 0
    for data_point in data.iterrows():
        data_point = data_point[1]
        max_x = np.max(data_point['corrected_xs']) # meters
        max_y = np.max(data_point['corrected_ys'])
        
        min_x = np.min(data_point['corrected_xs'])
#         min_x = np.ceil(min_x / 0.2) if min_x > 0 else np.floor(min_x / 0.2)
        
        min_y = np.min(data_point['corrected_ys'])
#         min_y = np.ceil(min_y / 0.2) if min_y > 0 else np.floor(min_y / 0.2)
        
        global_max_x = max_x if max_x > global_max_x else global_max_x
        global_max_y = max_y if max_y < global_max_y else global_max_y
        global_min_x = min_x if min_x > global_min_x else global_min_x
        global_min_y = min_y if min_y < global_min_y else global_min_y
        
    NED_bounds = [global_min_y, global_min_x, global_max_y, global_max_x]
    hight = int(np.ceil((global_max_y - global_min_y) / 0.2))
    width = int(np.ceil((global_max_x - global_min_x) / 0.2))
    print(hight)
    print(width)
    grid = np.zeros((hight, width))
    # min must be less then zero
    return grid, NED_bounds


def occupancy_map(data, res=0.2):
    grid, [min_y, min_x, max_y, max_x] = initialize_grid(data, res)
    initial_location = [-np.round(min_y / 0.2), -np.round(min_x / 0.2)]
    for data_point in data.iterrows():
        data_point = data_point[1]
        thetas = np.arctan2(data_point['corrected_ys'], data_point['corrected_xs'])
        ranges = np.sqrt(data_point['corrected_ys'] ** 2 + data_point['corrected_xs'] ** 2)
        
    return grid
#         local_max_x = np.max(data_point['corrected_xs']) # meters
#         local_max_y = np.max(data_point['corrected_ys'])
#         local_min_x = np.min(data_point['corrected_xs'])
#         local_min_y = np.min(data_point['corrected_ys'])
#         local_grid = [np.round(local_min_y / 0.2), np.round(local_max_y / 0.2), np.round(local_min_x / 0.2), np.round(local_max_x / 0.2)]
    
# occupancy_map(data)

# def reverse_sensor_model(grid, measurement):
#     pass

In [113]:
grid = occupancy_map(data)

476
581


In [125]:
cell_x, cell_y = np.meshgrid(np.array(range(grid.shape[0])), np.array(range(grid.shape[1])))
cell_theta = np.arctan2(cell_y, cell_x)
cell_ranges = np.sqrt(cell_y ** 2 + cell_x ** 2)

In [89]:
data = initialize(data_path)
# next(data.iterrows())[1]['velodyne_points']
# data[0]

476.0
581.0


TypeError: 'numpy.float64' object cannot be interpreted as an integer

In [7]:
pd.set_option('display.max_rows', data.shape[0]+1)
print(data)

NameError: name 'data' is not defined

In [325]:
print(data.iloc[0]['lat'])
print(data.iloc[0]['lon'])

49.009947690751
8.4234877923254


In [17]:
initial_coords = [data.iloc[0]['lat'], data.iloc[0]['lon'], data.iloc[0]['alt']]
current_coords = [data.iloc[-1]['lat'], data.iloc[-1]['lon'], data.iloc[-1]['alt']]
diff = navpy.lla2ned(*(current_coords + initial_coords))
print(diff)

[112.72550667 -16.03461898   0.72956336]


In [330]:
c, s = np.cos(np.array(data['yaw'])), np.sin(np.array(data['yaw']))
local_NE_rotation = np.array(((c, -s), (s, c)))

In [15]:
print(diff)

[112.72550667 -16.03461898   0.72956336]


In [ ]:
for i in range(200):
    reoriented_points = local_NE_rotation[:,:,i] @ np.vstack([np.array((data['velo_xs'][i])), np.array((data['velo_ys'][i]))])
    fig = px.scatter(x=reoriented_points[0], y=reoriented_points[1])
    fig.update_layout(
        width=1400,
        height=2000
    )
    fig.show()

In [352]:
np.arctan(diff[1] / diff[0])

-0.14129692309124703

In [314]:
float(data.iloc[0]['lat'])

49.009947690751